In [10]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
!pip install -U bitsandbytes -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.0 MB/s eta 0:00:00:00:0100:01


In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_API_KEY")



In [13]:
from huggingface_hub import login

In [14]:
login(secret_value_0)

In [15]:
!pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# 1. Imports and Libraries

In [ ]:
import pandas as pd
import glob
import json
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import evaluate

# 2. Loading and Extracting Data from JSON Papers

In [ ]:

def load_papers(directory="/kaggle/input/assignementdataset"):
    json_files = [f for f in Path(directory).glob("*.json") if f.is_file()]
    all_data = []
    for file in tqdm(json_files, desc="Loading JSON files"):
        with open(file, 'r') as f:
            try:
                doc = json.load(f)
                text = extract_text_from_json(doc)
                all_data.append({"file": file.name, "text": text})
            except json.JSONDecodeError as e:
                print(f"Error parsing {file.name}: {e}")
                continue
    return pd.DataFrame(all_data)

def extract_text_from_json(doc):
    if "body_text" in doc:
        return " ".join([section["text"] for section in doc["body_text"] if "text" in section])
    elif isinstance(doc, dict):
        return " ".join(str(v) for v in doc.values() if isinstance(v, str))
    return ""


# 3. Splitting the QA Dataset into Training and Testing

In [ ]:
def split_qa_dataset(qa_df, test_size=0.2, random_state=42):
    train_df, test_df = train_test_split(qa_df, test_size=test_size, random_state=random_state)
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

# 4. Building the Vector Retriever for Context Matching

In [ ]:
def build_retriever(df, model_name="all-MiniLM-L6-v2"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SentenceTransformer(model_name, device=device)
    texts = df["text"].fillna("").tolist()
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=16)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.astype(np.float32))
    return model, index

# 5. Preparing the QA Dataset for Fine-Tuning

In [ ]:
def prepare_finetuning_dataset(qa_df):
    def format_example(row):
        prompt = f"""
### Question: {row['question']}
### Context: {row['context'][:2000]}
### Instructions:
Answer: {row['answer']}
Context Snippet: {row['context']}
Location: {row['context_location']}
"""
        return {"text": prompt}
    
    dataset = Dataset.from_pandas(qa_df)
    dataset = dataset.map(format_example)
    return dataset

# 6. Fine-Tuning the Mistral Model with LoRA

In [ ]:
# Fine-tune Mistral model
def finetune_mistral(qa_train_df, output_dir="mistral_finetuned"):
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
    
    # Apply LoRA for efficient fine-tuning
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    
    # Prepare dataset
    train_dataset = prepare_finetuning_dataset(qa_train_df)
    
    # Tokenize dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    train_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        evaluation_strategy="no",
        report_to="none"
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset
    )
    
    trainer.train()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    return model, tokenizer


# 7. Retrieving and Generating Answers with the Fine-Tuned Model

In [ ]:
# Retrieve and generate answer with fine-tuned model
def retrieve_and_answer(question, paper_df, retriever_model, index, finetuned_model, tokenizer, max_context_length=2000):
    question_embedding = retriever_model.encode([question])[0]
    distances, indices = index.search(np.array([question_embedding]).astype(np.float32), k=1)
    paper_idx = indices[0][0]
    relevant_paper = paper_df.iloc[paper_idx]
    context = relevant_paper["text"][:max_context_length]
    
    prompt = f"""
You are a highly capable AI assistant trained in reproductive medicine literature. Your task is to reason carefully and answer the question based on the provided academic context.

### Instructions:
- Read the context below.
- Infer and construct the most accurate answer possible.
- Identify and copy the exact sentence or paragraph that supports your answer.
- Specify its location (e.g., sentence number or paragraph number).

### Format:
Answer: <your well-reasoned answer>
Context Snippet: <exact supporting sentence or paragraph from the context>
Location: <where the snippet is found — e.g., "2nd sentence", "3rd paragraph", etc.>

### Input:
Question: {question}
Context: {context}
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to("cuda")
    outputs = finetuned_model.generate(**inputs, max_new_tokens=500, temperature=0.2)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Parse the output
    answer = context_snippet = location = "Not found"
    for line in response_text.split("\n"):
        if line.strip().lower().startswith("answer:"):
            answer = line.split(":", 1)[1].strip()
        elif line.strip().lower().startswith("context snippet:"):
            context_snippet = line.split(":", 1)[1].strip()
        elif line.strip().lower().startswith("location:"):
            location = line.split(":", 1)[1].strip()
    
    return {
        "paper_title": relevant_paper["file"],
        "answer": answer,
        "context_snippet": context_snippet,
        "location": location,
        "paper_idx": paper_idx
    }


# 8. Evaluating Model Predictions

In [ ]:
# Evaluate predictions against true labels
def evaluate_predictions(test_df, paper_df, retriever_model, index, finetuned_model, tokenizer):
    results = []
    exact_match = evaluate.load("exact_match")
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating test set"):
        question = row["question"]
        true_labels = {
            "paper_title": row["paper"],
            "answer": row["answer"],
            "context_snippet": row["context"],
            "location": row["context_location"]
        }
        
        pred = retrieve_and_answer(question, paper_df, retriever_model, index, finetuned_model, tokenizer)
        
        # Evaluate
        paper_correct = pred["paper_title"] == true_labels["paper_title"]
        answer_em = exact_match.compute(predictions=[pred["answer"]], references=[true_labels["answer"]])["exact_match"]
        context_em = exact_match.compute(predictions=[pred["context_snippet"]], references=[true_labels["context_snippet"]])["exact_match"]
        location_em = exact_match.compute(predictions=[pred["location"]], references=[str(true_labels["location"])])["exact_match"]
        
        results.append({
            "question": question,
            "pred_paper": pred["paper_title"],
            "true_paper": true_labels["paper_title"],
            "paper_correct": paper_correct,
            "pred_answer": pred["answer"],
            "true_answer": true_labels["answer"],
            "answer_em": answer_em,
            "pred_context": pred["context_snippet"],
            "true_context": true_labels["context_snippet"],
            "context_em": context_em,
            "pred_location": pred["location"],
            "true_location": true_labels["location"],
            "location_em": location_em
        })
    
    results_df = pd.DataFrame(results)
    print("\nEvaluation Summary:")
    print(f"Paper Retrieval Accuracy: {results_df['paper_correct'].mean():.4f}")
    print(f"Answer Exact Match: {results_df['answer_em'].mean():.4f}")
    print(f"Context Exact Match: {results_df['context_em'].mean():.4f}")
    print(f"Location Exact Match: {results_df['location_em'].mean():.4f}")
    
    return results_df


# 9. Main Execution

In [17]:

if __name__ == "__main__":

    print("Loading paper dataset...")
    paper_df = load_papers()
    print(f"Loaded {len(paper_df)} papers.")
    
    print("Loading QA dataset...")
    qa_df = pd.read_csv("/kaggle/input/assignmentqa/train_data(2).csv")  
    print(f"Loaded {len(qa_df)} QA pairs.")
    
    print("Splitting QA dataset...")
    qa_train_df, qa_test_df = split_qa_dataset(qa_df)
    print(f"Training set: {len(qa_train_df)} QA pairs")
    print(f"Test set: {len(qa_test_df)} QA pairs")
    
    print("Fine-tuning Mistral model...")
    finetuned_model, tokenizer = finetune_mistral(qa_train_df)
   
    print("Building retriever...")
    retriever_model, index = build_retriever(paper_df)
 
    print("Evaluating on test set...")
    results_df = evaluate_predictions(qa_test_df, paper_df, retriever_model, index, finetuned_model, tokenizer)
   
    results_df.to_csv("evaluation_results.csv", index=False)
    print("Results saved to evaluation_results.csv")

Loading paper dataset...


Loading JSON files: 100%|██████████| 43/43 [00:00<00:00, 470.89it/s]

Loaded 43 papers.
Loading QA dataset...
Loaded 215 QA pairs.
Splitting QA dataset...
Training set: 172 QA pairs
Test set: 43 QA pairs
Fine-tuning Mistral model...


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [6]:
import pandas as pd
import json
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
from pathlib import Path
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split

# Load JSON papers
def load_papers(directory="/kaggle/input/assignementdataset"):
    directory_path = Path(directory)
    if not directory_path.exists():
        raise FileNotFoundError(f"Directory {directory} does not exist. Please check the path.")
    
    json_files = [f for f in directory_path.glob("*.json") if f.is_file()]
    if not json_files:
        raise ValueError(f"No JSON files found in {directory}. Please ensure the directory contains valid JSON files.")
    
    all_data = []
    for file in tqdm(json_files, desc="Loading JSON files"):
        with open(file, 'r') as f:
            try:
                doc = json.load(f)
                text = extract_text_from_json(doc)
                if not text.strip():
                    print(f"Warning: Empty text extracted from {file.name}")
                    continue
                all_data.append({"file": file.name, "text": text})
            except json.JSONDecodeError as e:
                print(f"Error parsing {file.name}: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error processing {file.name}: {e}")
                continue
    
    papers_df = pd.DataFrame(all_data)
    if papers_df.empty:
        raise ValueError("No valid papers loaded. Check JSON files for content or parsing errors.")
    
    print(f"Loaded {len(papers_df)} papers successfully.")
    return papers_df

def extract_text_from_json(doc):
    if "body_text" in doc:
        return " ".join([section["text"] for section in doc["body_text"] if "text" in section])
    elif isinstance(doc, dict):
        return " ".join(str(v) for v in doc.values() if isinstance(v, str))
    return ""

# Load and prepare QA dataset
def load_qa_dataset(csv_path="/kaggle/input/assignmentqa/train_data(2).csv"):
    if not Path(csv_path).exists():
        raise FileNotFoundError(f"CSV file {csv_path} not found.")
    
    df = pd.read_csv(csv_path)
    required_columns = ["question", "answer", "context", "context_location", "paper"]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"CSV missing required columns: {missing_columns}")
    
    if df.empty:
        raise ValueError("CSV file is empty. Please provide a valid dataset.")
    
    print(f"Loaded {len(df)} QA pairs from {csv_path}.")
    return df

# Build vector retriever
def build_retriever(df, model_name="all-MiniLM-L6-v2"):
    if df.empty:
        raise ValueError("Papers DataFrame is empty. Cannot build retriever.")
    
    model = SentenceTransformer(model_name, device="cuda" if torch.cuda.is_available() else "cpu")
    texts = df["text"].fillna("").tolist()
    if not any(texts):
        raise ValueError("All paper texts are empty. Cannot build embeddings.")
    
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=16)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.astype(np.float32))
    return model, index

# Prepare training dataset
def prepare_training_data(qa_df, papers_df, max_context_length=2000):
    data = []
    skipped_rows = []
    
    # Normalize filenames for matching
    papers_df["file_normalized"] = papers_df["file"].str.replace(r"\.json$", "", regex=True)
    qa_df["paper_normalized"] = qa_df["paper"].str.replace(r"\.json$", "", regex=True)
    
    for idx, row in tqdm(qa_df.iterrows(), desc="Preparing training data", total=len(qa_df)):
        question = row["question"]
        answer = row["answer"]
        context_snippet = row["context"]
        location = row["context_location"]
        paper_name = row["paper_normalized"]
        
        # Retrieve full paper text
        paper_row = papers_df[papers_df["file_normalized"] == paper_name]
        if paper_row.empty:
            skipped_rows.append((idx, paper_name, "Paper not found in papers_df"))
            continue
        
        context = paper_row.iloc[0]["text"][:max_context_length]
        if not context.strip():
            skipped_rows.append((idx, paper_name, "Empty paper text"))
            continue
        
        # Create prompt
        prompt = f"""
You are a highly capable AI assistant trained in reproductive medicine literature. Your task is to answer the question based on the provided academic context and identify the supporting context snippet and its location.

### Input:
Question: {question}
Context: {context}

### Output:
Answer: {answer}
Context Snippet: {context_snippet}
Location: {location}
"""
        data.append({
            "prompt": prompt,
            "output": f"Answer: {answer}\nContext Snippet: {context_snippet}\nLocation: {location}"
        })
    
    # Log skipped rows
    if skipped_rows:
        print(f"Skipped {len(skipped_rows)} rows due to issues:")
        for idx, paper_name, reason in skipped_rows[:10]:  # Limit to first 10 for brevity
            print(f"Row {idx}, Paper: {paper_name}, Reason: {reason}")
        if len(skipped_rows) > 10:
            print(f"... and {len(skipped_rows) - 10} more skipped rows.")
    
    if not data:
        raise ValueError("No valid training examples created. Check paper filenames and data integrity.")
    
    print(f"Created {len(data)} valid training examples.")
    return Dataset.from_list(data)

# Split dataset
def split_dataset(dataset, test_size=0.2, random_state=42):
    if len(dataset) == 0:
        raise ValueError("Dataset is empty. Cannot perform train-test split.")
    
    train_data, val_data = train_test_split(dataset, test_size=test_size, random_state=random_state)
    print(f"Split dataset: {len(train_data)} training, {len(val_data)} validation examples.")
    
    if not train_data or not val_data:
        raise ValueError("Train or validation set is empty after split. Adjust test_size or check dataset.")
    
    return Dataset.from_list(train_data), Dataset.from_list(val_data)

# Fine-tune Mistral 7B
def fine_tune_mistral(train_dataset, val_dataset, model_name="mistralai/Mixtral-7B-Instruct-v0.1", output_dir="./mistral_finetuned"):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")

    # Apply LoRA
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)

    # Tokenize dataset
    def tokenize_function(examples):
        inputs = tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=2048)
        outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=512)
        inputs["labels"] = outputs["input_ids"]
        return inputs

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_val = val_dataset.map(tokenize_function, batched=True)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_steps=100,
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=50,
        report_to="none"
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val
    )

    # Train the model
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    return model, tokenizer

# Inference with fine-tuned model
def retrieve_and_answer_finetuned(question, papers_df, retriever_model, index, finetuned_model, tokenizer, max_context_length=2000):
    # Retrieve relevant paper
    question_embedding = retriever_model.encode([question])[0]
    distances, indices = index.search(np.array([question_embedding]).astype(np.float32), k=1)
    paper_idx = indices[0][0]
    relevant_paper = papers_df.iloc[paper_idx]
    context = relevant_paper["text"][:max_context_length]

    # Create prompt
    prompt = f"""
You are a highly capable AI assistant trained in reproductive medicine literature. Your task is to answer the question based on the provided academic context and identify the supporting context snippet and its location.

### Input:
Question: {question}
Context: {context}

### Output:
Answer: 
Context Snippet: 
Location: 
"""
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to("cuda")
    outputs = finetuned_model.generate(**inputs, max_new_tokens=500, temperature=0.2)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Parse response
    answer = context_snippet = location = "Not found"
    for line in response.split("\n"):
        if line.strip().lower().startswith("answer:"):
            answer = line.split(":", 1)[1].strip()
        elif line.strip().lower().startswith("context snippet:"):
            context_snippet = line.split(":", 1)[1].strip()
        elif line.strip().lower().startswith("location:"):
            location = line.split(":", 1)[1].strip()

    return {
        "paper_title": relevant_paper["file"],
        "answer": answer,
        "context_snippet": context_snippet,
        "location": location,
        "paper_idx": paper_idx
    }

# Main execution
if __name__ == "__main__":
    print("Loading papers...")
    papers_df = load_papers()
    
    print("Loading QA dataset...")
    qa_df = load_qa_dataset()
    
    print("Building retriever...")
    retriever_model, index = build_retriever(papers_df)
    
    print("Preparing training data...")
    dataset = prepare_training_data(qa_df, papers_df)
    train_dataset, val_dataset = split_dataset(dataset)
    
    print("Fine-tuning Mistral 7B...")
    finetuned_model, finetuned_tokenizer = fine_tune_mistral(train_dataset, val_dataset)
    
    print("Testing fine-tuned model...")
    custom_question = input("\nEnter your question: ")
    result = retrieve_and_answer_finetuned(custom_question, papers_df, retriever_model, index, finetuned_model, finetuned_tokenizer)
    
    print("\nResults:")
    print(f"Most Relevant Paper: {result['paper_title']}")
    print(f"Answer: {result['answer']}")
    print(f"Context Snippet: {result['context_snippet']}")
    print(f"Location: {result['location']}")

Loading papers...


Loading JSON files: 100%|██████████| 43/43 [00:00<00:00, 344.26it/s]

Loaded 43 papers successfully.
Loading QA dataset...
Loaded 215 QA pairs from /kaggle/input/assignmentqa/train_data(2).csv.
Building retriever...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Preparing training data...


Preparing training data: 100%|██████████| 215/215 [00:00<00:00, 2913.23it/s]

Skipped 215 rows due to issues:
Row 0, Paper: Awareness-knowledge-and-misconceptions-of-adoles, Reason: Paper not found in papers_df
Row 1, Paper: Awareness-knowledge-and-misconceptions-of-adoles, Reason: Paper not found in papers_df
Row 2, Paper: Awareness-knowledge-and-misconceptions-of-adoles, Reason: Paper not found in papers_df
Row 3, Paper: Awareness-knowledge-and-misconceptions-of-adoles, Reason: Paper not found in papers_df
Row 4, Paper: Awareness-knowledge-and-misconceptions-of-adoles, Reason: Paper not found in papers_df
Row 5, Paper: A-novel-approach-using-vaginal-natural-orifice-tra, Reason: Paper not found in papers_df
Row 6, Paper: A-novel-approach-using-vaginal-natural-orifice-tra, Reason: Paper not found in papers_df
Row 7, Paper: A-novel-approach-using-vaginal-natural-orifice-tra, Reason: Paper not found in papers_df
Row 8, Paper: A-novel-approach-using-vaginal-natural-orifice-tra, Reason: Paper not found in papers_df
Row 9, Paper: A-novel-approach-using-vaginal-natura

ValueError: No valid training examples created. Check paper filenames and data integrity.